# Coursera Capstone Final Project

I want to open a new Restaurant in Porto District with the district code in Portugal of 4. So I want to decide which is the best municipality place to open the new restaurant. My criteria is based on the amount of restaurants in each municipality vs. population density in each municipality.

In [8]:
#import the packages
import pandas as pd

In [9]:
#creates a source for the Beautiful Soup
df = pd.read_excel(r'pordata.xlsx')
df = df[['Region','Municipality','Density']]
# Add Postal code , latitude and longitude collumn
df["Municipality code"] = ""
df["Latitude"] = ""
df["Longitude"] = ""

df.head()

,Region,Municipality,Density,Municipality code,Latitude,Longitude
0,Alto Minho,Arcos de Valdevez,47,,,
1,Alto Minho,Caminha,116.4,,,
2,Alto Minho,Melgaço,34.4,,,
3,Alto Minho,Monção,85.1,,,
4,Alto Minho,Paredes de Coura,62.1,,,


In [15]:
# All districts and municipalities of Portugal
code_data = pd.read_csv("concelhos.csv")[["nome_concelho","cod_distrito","cod_concelho"]]
# Porto district code is 4
porto_data = code_data[code_data['cod_distrito'] == 4]
porto_data.head()

porto_data['nome_concelho'] 
df['Municipality']

#geo_data[['']]


47            Miranda do Douro
48                   Mirandela
49                   Mogadouro
50           Torre de Moncorvo
51                   Vila Flor
52                     Vimioso
53                     Vinhais
54             Alfândega da Fé
55                    Bragança
56        Carrazeda de Ansiães
57    Freixo de Espada à Cinta
58        Macedo de Cavaleiros
Name: nome_concelho, dtype: object

In [28]:
# Json file to check latitude and longitude based on Postal Code

http://codigospostais.appspot.com/cp4?

,0
0,English Version //<![CDATA[ OsInputPrompt_Boot...


In [6]:
#Change the neighbourhood to the borough
df.at[8, 'Neighbourhood'] = "Queen's Park"
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [7]:
#Combine rows with multiple neighbourhoods (same postal code)
postcodeGroups = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)

In [8]:
postcodeGroups.head(5)

Postcode  Borough    
M1B       Scarborough                            [Rouge, Malvern]
M1C       Scarborough    [Highland Creek, Rouge Hill, Port Union]
M1E       Scarborough         [Guildwood, Morningside, West Hill]
M1G       Scarborough                                    [Woburn]
M1H       Scarborough                                 [Cedarbrae]
Name: Neighbourhood, dtype: object

In [9]:
#Turn this into a new dataframe.
dfPCG = pd.DataFrame(postcodeGroups)
dfPCG = dfPCG.reset_index(['Postcode', 'Borough'])
dfPCG.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [10]:
# Print the shape
dfPCG.shape

(103, 3)

In [11]:
#columns for the latitude and longitude of the postal codes
dfPCG["Latitude"] = ""
dfPCG["Longitude"] = ""
dfPCG.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",,
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",,
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",,
3,M1G,Scarborough,[Woburn],,
4,M1H,Scarborough,[Cedarbrae],,


In [12]:
#Import the table with longitude and latitude
geo_data = pd.read_csv("https://cocl.us/Geospatial_data")
geo_data.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# Include on the datafraem the geo_data
dfPCG["Latitude"] = geo_data["Latitude"]
dfPCG["Longitude"] = geo_data["Longitude"]
dfPCG.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


In [14]:
# import other packages
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [15]:
# Create an address for Toronto.
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


In [16]:
# create a  map of Toronto using latitude and longitude values and Folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfPCG['Latitude'], dfPCG['Longitude'], dfPCG['Borough'], dfPCG['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
# function to find venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        CLIENT_ID = 'ZC4GEQ0R3WUBDVREK3PBHNT0O3HVJFKF1TFYQPIQEINCPREG'
        CLIENT_SECRET = 'EWPJRXRGGNTDSJKXCLT2XAA4SLIJLC0IERAF4YHNM3MKFYRC'
        VERSION = '20180604'
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# call function
toronto_venues = getNearbyVenues(names=dfPCG['Postcode'],
                                   latitudes=dfPCG['Latitude'],
                                   longitudes=dfPCG['Longitude']
                                  )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(2259, 7)


,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1B,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,M1C,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [24]:
#A count of venues for each postal code.
toronto_venues.groupby('Postcode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M1B,2,2,2,2,2,2
M1C,2,2,2,2,2,2
M1E,9,9,9,9,9,9
M1G,4,4,4,4,4,4
M1H,8,8,8,8,8,8
M1J,1,1,1,1,1,1
M1K,4,4,4,4,4,4
M1L,10,10,10,10,10,10
M1M,3,3,3,3,3,3


In [30]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M1C,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,M1E,Intersection,Spa,Electronics Store,Pizza Place,Breakfast Spot,Tech Startup,Medical Center,Rental Car Location,Mexican Restaurant,Dog Run
3,M1G,Coffee Shop,Convenience Store,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Hakka Restaurant,Bakery,Lounge,Caribbean Restaurant,Athletics & Sports,Bank,Fried Chicken Joint,Thai Restaurant,Yoga Studio,Drugstore


In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_onehot.shape
# group
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head()
toronto_grouped.shape

(100, 281)

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0            Print Shop   0.5
1  Fast Food Restaurant   0.5
2    Miscellaneous Shop   0.0
3         Movie Theater   0.0
4                 Motel   0.0


----M1C----
                        venue  freq
0  Construction & Landscaping   0.5
1                         Bar   0.5
2           Accessories Store   0.0
3           Mobile Phone Shop   0.0
4               Movie Theater   0.0


----M1E----
                venue  freq
0                 Spa  0.11
1        Intersection  0.11
2      Breakfast Spot  0.11
3  Mexican Restaurant  0.11
4   Electronics Store  0.11


----M1G----
                        venue  freq
0                 Coffee Shop  0.50
1           Convenience Store  0.25
2           Korean Restaurant  0.25
3           Accessories Store  0.00
4  Modern European Restaurant  0.00


----M1H----
                  venue  freq
0   Fried Chicken Joint  0.12
1      Hakka Restaurant  0.12
2  Caribbean Restaurant  0.12
3       Thai Restaurant  0.12
4 

            venue  freq
0     Pizza Place  0.11
1    Dessert Shop  0.08
2  Sandwich Place  0.08
3            Café  0.05
4      Restaurant  0.05


----M4T----
                venue  freq
0        Tennis Court   0.5
1          Playground   0.5
2   Accessories Store   0.0
3  Miscellaneous Shop   0.0
4       Movie Theater   0.0


----M4V----
                 venue  freq
0                  Pub  0.14
1          Coffee Shop  0.14
2           Bagel Shop  0.07
3  American Restaurant  0.07
4     Sushi Restaurant  0.07


----M4W----
               venue  freq
0               Park  0.50
1              Trail  0.25
2         Playground  0.25
3  Accessories Store  0.00
4              Motel  0.00


----M4X----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.07
2         Park  0.04
3         Café  0.04
4       Bakery  0.04


----M4Y----
                 venue  freq
0  Japanese Restaurant  0.07
1          Coffee Shop  0.07
2     Sushi Restaurant  0.06
3              Gay Bar  0.03
4          

               venue  freq
0           Pharmacy  0.17
1         Beer Store  0.17
2       Liquor Store  0.17
3               Café  0.17
4  Convenience Store  0.17


----M9L----
                 venue  freq
0          Pizza Place  0.33
1  Empanada Restaurant  0.33
2             Pharmacy  0.33
3         Home Service  0.00
4          Music Store  0.00


----M9M----
               venue  freq
0     Baseball Field   1.0
1  Accessories Store   0.0
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----M9N----
                venue  freq
0                Park   1.0
1   Accessories Store   0.0
2  Miscellaneous Shop   0.0
3       Movie Theater   0.0
4               Motel   0.0


----M9P----
                venue  freq
0         Pizza Place  0.33
1      Sandwich Place  0.17
2        Intersection  0.17
3  Chinese Restaurant  0.17
4         Coffee Shop  0.17


----M9R----
               venue  freq
0        Pizza Place  0.25
1  Mobile Phone Shop  0.25
2              

In [32]:
# number of clusters
kclusters = 5

toronto_grouped_cl = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_cl)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 2,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 4, 0, 1, 0, 0, 4, 2, 0, 2, 0, 0], dtype=int32)

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels New', kmeans.labels_)
toronto_merged = dfPCG
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')
toronto_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels New,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,0.0,Intersection,Spa,Electronics Store,Pizza Place,Breakfast Spot,Tech Startup,Medical Center,Rental Car Location,Mexican Restaurant,Dog Run
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,0.0,Coffee Shop,Convenience Store,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,0.0,Hakka Restaurant,Bakery,Lounge,Caribbean Restaurant,Athletics & Sports,Bank,Fried Chicken Joint,Thai Restaurant,Yoga Studio,Drugstore
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476,0.0,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029,0.0,Department Store,Playground,Discount Store,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577,0.0,Bus Line,Bakery,Park,Soccer Field,Metro Station,Bus Station,Intersection,Fast Food Restaurant,Convenience Store,Farmers Market
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476,0.0,Motel,American Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848,0.0,College Stadium,General Entertainment,Skating Rink,Café,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters